# Méthode KNN

La méthode KNN est un algorithme d'apprentissage supervisé. Il vise à …

Dans un 1er temps nous allons importer les données nécessaires.

# Initialisation 
Initialisation des bibliothèques nécessaires

In [258]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

# Récupération des données

In [259]:
data_directory = '../input/gem-predictive-maintenance-2022'
chosen_dataset = 'FD003'

df_train = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_train.csv'), index_col=0) 
df_test = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_test.csv'), index_col=0)

df_train

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,2388,100.0,39.11,23.3537,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,2388,100.0,38.99,23.4491,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,2388,100.0,38.85,23.3669,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,2388,100.0,38.96,23.2951,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,2388,100.0,39.14,23.4583,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,2388,100.0,38.44,22.9631,NaN,NaN,NaN,NaN,NaN,NaN
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,2388,100.0,38.50,22.9746,NaN,NaN,NaN,NaN,NaN,NaN
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,2388,100.0,38.39,23.0682,NaN,NaN,NaN,NaN,NaN,NaN
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,2388,100.0,38.31,23.0753,NaN,NaN,NaN,NaN,NaN,NaN


# Testing set

In [260]:
df_train.describe()

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27
count,24720.000000,24720.000000,24720.000000,24720.000000,24720.0,24720.00,24720.000000,24720.000000,24720.000000,2.472000e+04,...,24720.0,24720.0,24720.000000,24720.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,48.631877,139.077063,-0.000024,0.000005,100.0,518.67,642.457858,1588.079175,1404.471212,1.462000e+01,...,2388.0,100.0,38.988552,23.393024,NaN,NaN,NaN,NaN,NaN,NaN
std,29.348985,98.846675,0.002194,0.000294,0.0,0.00,0.523031,6.810418,9.773178,3.552786e-15,...,0.0,0.0,0.248865,0.149234,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,1.000000,-0.008600,-0.000600,100.0,518.67,640.840000,1564.300000,1377.060000,1.462000e+01,...,2388.0,100.0,38.170000,22.872600,NaN,NaN,NaN,NaN,NaN,NaN
25%,23.000000,62.000000,-0.001500,-0.000200,100.0,518.67,642.080000,1583.280000,1397.187500,1.462000e+01,...,2388.0,100.0,38.830000,23.296200,NaN,NaN,NaN,NaN,NaN,NaN
50%,47.000000,124.000000,0.000000,-0.000000,100.0,518.67,642.400000,1587.520000,1402.910000,1.462000e+01,...,2388.0,100.0,38.990000,23.391600,NaN,NaN,NaN,NaN,NaN,NaN
75%,74.000000,191.000000,0.001500,0.000300,100.0,518.67,642.790000,1592.412500,1410.600000,1.462000e+01,...,2388.0,100.0,39.140000,23.483300,NaN,NaN,NaN,NaN,NaN,NaN
max,100.000000,525.000000,0.008600,0.000700,100.0,518.67,645.110000,1615.390000,1441.160000,1.462000e+01,...,2388.0,100.0,39.850000,23.950500,NaN,NaN,NaN,NaN,NaN,NaN


# Add the rule

In [261]:
def add_rul(g):
    g['RUL'] = max(g['time_in_cycles']) - g['time_in_cycles']
    return g

df_train = df_train.groupby('engine_no').apply(add_rul)
df_test = df_test.groupby('engine_no').apply(add_rul)


print (df_train)

       engine_no  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0              1               1       -0.0005        0.0004         100.0   
1              1               2        0.0008       -0.0003         100.0   
2              1               3       -0.0014       -0.0002         100.0   
3              1               4       -0.0020        0.0001         100.0   
4              1               5        0.0016        0.0000         100.0   
...          ...             ...           ...           ...           ...   
24715        100             148       -0.0016       -0.0003         100.0   
24716        100             149        0.0034       -0.0003         100.0   
24717        100             150       -0.0016        0.0004         100.0   
24718        100             151       -0.0023        0.0004         100.0   
24719        100             152        0.0000        0.0003         100.0   

       sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  ...  s

# Définition des valeurs

In [262]:
#Y :
target_column =['RUL']

#X :
selected_features = [ 'op_setting_1', 'op_setting_2',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8',
       'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21']

In [263]:
x_train = df_train[selected_features].values.astype('int')
y_train = df_train[target_column].values.ravel()

x_test = df_test[selected_features].values.astype('int')
y_test = df_test[target_column].values.ravel()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(24720, 16)
(24720,)
(16596, 16)
(16596,)


In [264]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)
print(y_pred.shape)

(16596,)


In [265]:


df_resultat_test = df_test.assign(RUL_predict = y_pred)
print(df_resultat_test)

       engine_no  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0              1               1       -0.0017       -0.0004         100.0   
1              1               2        0.0006       -0.0002         100.0   
2              1               3        0.0014       -0.0003         100.0   
3              1               4        0.0027        0.0001         100.0   
4              1               5       -0.0001        0.0001         100.0   
...          ...             ...           ...           ...           ...   
16591        100             243        0.0011       -0.0003         100.0   
16592        100             244       -0.0024        0.0002         100.0   
16593        100             245       -0.0053        0.0002         100.0   
16594        100             246       -0.0006        0.0001         100.0   
16595        100             247        0.0023        0.0004         100.0   

       sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  ...  s

In [266]:
masque = ['engine_no', 'RUL_predict', 'RUL']
df_final_test = df_resultat_test[masque]
print(df_final_test)

       engine_no  RUL_predict  RUL
0              1      207.875  232
1              1      209.595  231
2              1      201.675  230
3              1      186.995  229
4              1      237.890  228
...          ...          ...  ...
16591        100       41.985    4
16592        100       42.825    3
16593        100       47.445    2
16594        100       39.920    1
16595        100       34.350    0

[16596 rows x 3 columns]


In [267]:
df_final = df_final_test.groupby('engine_no').agg({'RUL_predict': 'first'}).reset_index()
print(df_final)

    engine_no  RUL_predict
0           1    207.87500
1           2    171.02625
2           3    164.88500
3           4    102.00000
4           5    349.60500
..        ...          ...
95         96    176.98000
96         97    193.97500
97         98    146.12000
98         99    217.47500
99        100    346.94500

[100 rows x 2 columns]


In [268]:
df_test_resultat = df_final
df_test_resultat ['RUL'] = df_resultat_test['RUL']
df_test_resultat ['difference'] = df_test_resultat ['RUL']- df_test_resultat['RUL_predict']
df_test_resultat_test = df_final_test
df_test_resultat_test ['RUL'] = df_resultat_test['RUL']
df_test_resultat_test ['difference'] = df_test_resultat_test ['RUL']- df_test_resultat_test['RUL_predict']
print(df_test_resultat)
print(df_test_resultat.describe())
print(df_test_resultat_test.describe())

    engine_no  RUL_predict  RUL  difference
0           1    207.87500  232    24.12500
1           2    171.02625  231    59.97375
2           3    164.88500  230    65.11500
3           4    102.00000  229   127.00000
4           5    349.60500  228  -121.60500
..        ...          ...  ...         ...
95         96    176.98000  137   -39.98000
96         97    193.97500  136   -57.97500
97         98    146.12000  135   -11.12000
98         99    217.47500  134   -83.47500
99        100    346.94500  133  -213.94500

[100 rows x 4 columns]
        engine_no  RUL_predict         RUL  difference
count  100.000000   100.000000  100.000000  100.000000
mean    50.500000   199.025217  182.500000  -16.525217
std     29.011492    60.584531   29.011492   69.845411
min      1.000000    67.960000  133.000000 -213.945000
25%     25.750000   155.117500  157.750000  -54.356250
50%     50.500000   193.425000  182.500000  -13.647500
75%     75.250000   224.918750  207.250000   37.427500
max    1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [269]:
df_final.columns = ['engine_no', 'RUL']
df_final[['engine_no', 'RUL']].to_csv('submission.csv', index=False)
df_final

ValueError: Length mismatch: Expected axis has 4 elements, new values have 2 elements